In [ ]:
! pip install requests beautifulsoup4 certifi pyopenssl urllib3 html5lib pandas tqdm sqlalchemy psycopg2

In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from pandas import read_csv
from datetime import datetime
from tqdm import tqdm
from sqlalchemy import create_engine, URL
from pathlib import Path
import json

In [ ]:
article = []
for i in range(1,455):
    data = requests.get(f"https://www.scholarship.in.th/category/news-update/page/{i}/")
    soup = BeautifulSoup(data.content, "html5lib")
    for ar in soup.find_all("div", {"class": "td_module_10 td_module_wrap td-animation-stack"}):
        href = ar.a["href"]
        article.append(href)

In [ ]:
with open("scholarship.txt", "w") as f:
    for a in set(article):
        f.write(f"{a}\n")

---------

In [ ]:
with open("scholarship.txt", "r") as f:
    article = list(map(str.strip, f.readlines()))

In [ ]:
db_data = {}
filename = "scholarship_{i}.csv"
for i, url in tqdm(enumerate(article), total=len(article)):
    if i<9000:
        continue
    if i%100 == 0 and len(db_data) > 0:
        df = DataFrame(db_data).transpose()
        df.to_csv(filename.format(i))
        db_data = {}
        
    try:
        data = requests.get(url)
        soup = BeautifulSoup(data.content, "html5lib")

        paragraph = []
        for p in soup.find("div", {"class": "td-post-content"})("p"):
            if "script" in [c.name for c in  p.findChildren()]:
                continue
            text = p.text.replace("\xa0", " ")
            if len(text) < 150:
                continue
            paragraph.append(text)

        db_data[url] = {
                "url": url,
                "category": "Education",
                "title": soup.title.string,
                "content": None,
                "paragraphs": json.dumps(paragraph, ensure_ascii=False),
                "raw_html": data.text,
                "raw_json": None,
                "remark": None,
                "created_at": datetime.now(),
            }
    except Exception as e:
        print(f"Exception on {url} by {e.args}")

df = DataFrame(db_data).transpose()
df.to_csv(filename.format(i+1))


In [ ]:
engine = create_engine()

In [ ]:
list_path = list(Path(".").glob("scholarship_*.csv"))
for path in tqdm(list_path):
    df = read_csv(path)
    df.drop(columns=["Unnamed: 0"], inplace=True)
    df.to_sql("article_raw_data", engine, index=False, if_exists='append', method="multi")